In [1]:
!pip install langchain langchain-experimental langchain-community langchain-openai openai chromadb pypdf sentence_transformers gradio langchain-together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

In [2]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.0 MB/s eta 0:00:00


In [3]:
!pip install -U langchain-huggingface


In [4]:
from PyPDF2 import PdfFileReader
from sentence_transformers import SentenceTransformer
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import Together

def setup_rag_model(pdf_path, together_api_key):
    # Load the PDF document
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # Extract text from each page and combine them into a single document
    document_text = " ".join([page.page_content for page in pages])

    # Create embeddings
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma.from_documents(pages, embedding_function)

    # Initialize the LLM
    llm = Together(
        model="meta-llama/Llama-2-70b-chat-hf",
        max_tokens=256,
        temperature=0,
        top_k=1,
        together_api_key=together_api_key
    )

    # Setup the retriever
    retriever = db.as_retriever(similarity_score_threshold=0.9)

    # Define the prompt template for Q&A
    question_generation_template = """Based on the context below, generate a relevant question:
    CONTEXT: {context}"""
    QUESTION_PROMPT = PromptTemplate(template=f"[INST] {question_generation_template} [/INST]", input_variables=["context"])

    answer_generation_template = """Please answer the following question based on the context provided:
    CONTEXT: {context}
    QUESTION: {question}"""
    ANSWER_PROMPT = PromptTemplate(template=f"[INST] {answer_generation_template} [/INST]", input_variables=["context", "question"])

    # Create the retrieval chain
    question_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        input_key='query',
        return_source_documents=True,
        chain_type_kwargs={"prompt": QUESTION_PROMPT},
        verbose=True
    )

    answer_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        input_key='query',
        return_source_documents=True,
        chain_type_kwargs={"prompt": ANSWER_PROMPT},
        verbose=True
    )

    return question_chain, answer_chain, document_text

def generate_question(chain, document_text):
    query = "Generate a question based on the document."
    response = chain({"query": query, "context": document_text})
    return response['result']

def answer_question(chain, document_text, question):
    response = chain({"query": question, "context": document_text})
    return response['result']

# Example usage
pdf_path = "/content/somatosensory 1.pdf"  # Replace with the correct path to your PDF document
together_api_key = "5c3aaa83cb623ccf29c3038870387e97d17f64112b00d431f55f6fc19e4d75ab"  # Your Together API key

# Validate the API key
if not together_api_key or together_api_key == "your_together_api_key":
    raise ValueError("Invalid Together API key provided. Please ensure you have a valid key.")

# Setup the RAG model
question_chain, answer_chain, document_text = setup_rag_model(pdf_path, together_api_key)

# Generate a question
question = generate_question(question_chain, document_text)
print(f"Generated Question: {question}")

# Answer the generated question
answer = answer_question(answer_chain, document_text, question)
print(f"Answer: {answer}")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 0.3. An updated version of the class exists in the langchain-together package and should be used instead. To use it run `pip install -U langchain-together` and import as `from langchain_together import Together`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.
Generated Question:   Based on the context provided, a relevant question could be:

What are the different types of receptors in the somatosensory system, and how do they contribute to our perception of touch, pressure, temperature, and pain?

This question is relevant because the passage discusses the various receptors in the skin and muscles that are responsible for detecting different types of sensory information, including rapidly adapting and slowly adapting receptors, as well as nociceptors. Understanding the different types of receptors and their functions can help us better understand how the somatosensory system works and how it contributes to our overall perception of the world around us.


> Entering new RetrievalQA chain...

> Finished chain.
Answer:   That's a great question! The passage discusses several types of receptors in the somatosensory system, including:

1. Rapidly adapting receptors: These receptors are fo